In [1]:
import json
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Video_Games.json.gz')

In [4]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3.0,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4.0,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1.0,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4.0,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN


In [11]:
df_new = df[['reviewText','overall']]

In [28]:
df_new['overall'].value_counts()

5.0    1487366
4.0     412413
1.0     311891
3.0     212346
2.0     141333
Name: overall, dtype: int64

In [13]:
df_new.head()

,reviewText,overall
0,I used to play this game years ago and loved i...,1.0
1,The game itself worked great but the story lin...,3.0
2,I had to learn the hard way after ordering thi...,4.0
3,The product description should state this clea...,1.0
4,I would recommend this learning game for anyon...,4.0


In [32]:
df_pos_300k = df_new[df_new['overall'] == 5][:300000]
df_neg_300k = df_new[df_new['overall'] == 1][:300000]
df_new_600k = pd.concat([df_pos_300k,df_neg_300k],ignore_index=True)
df_new_600k['sentiment'] = df_new_600k['overall'].map({5.0:'pos',1.0:'neg'})
df_new_600k = df_new_600k[['reviewText','sentiment']]

In [33]:
df_new_600k

,reviewText,sentiment
0,Choose your career which sets your money for t...,pos
1,It took a few hours to get this up and running...,pos
2,I oredered this for a daughter who is now 33 a...,pos
3,Well I thought since this idem didn't have any...,pos
4,I got this to show my kids what I used to play...,pos
...,...,...
599995,Item broke after one charging cycle . when I r...,neg
599996,Terrible I don't recommend,neg
599997,Very poor quality. They do not hold a charge. ...,neg
599998,it broke :( where you plug the USB to charge. ...,neg


In [34]:
df_new_600k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   reviewText  599822 non-null  object
 1   sentiment   600000 non-null  object
dtypes: object(2)
memory usage: 9.2+ MB


In [36]:
df_new_600k['sentiment'].value_counts()

pos    300000
neg    300000
Name: sentiment, dtype: int64

In [37]:
df_new_600k.to_csv('games_600k.csv',index=False)